In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms


from collections import namedtuple
from sklearn.metrics import classification_report

# **Clone git để lấy dữ liệu**


In [3]:
!git clone https://github.com/ttlong13022811/ML-Echocardiogram-Analysis
traindir = "/content/ML-Echocardiogram-Analysis/DATA_CHAMBER_2021/train"
testdir = "/content/ML-Echocardiogram-Analysis/DATA_CHAMBER_2021/test"

Cloning into 'ML-Echocardiogram-Analysis'...
remote: Enumerating objects: 8364, done.
remote: Total 8364 (delta 0), reused 0 (delta 0), pack-reused 8364
Receiving objects: 100% (8364/8364), 488.03 MiB | 35.34 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (8328/8328), done.


# **Chuẩn bị dataset**

In [4]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def resize_crop_test(size):
  resize = transforms.Resize((size,size))
  crop = transforms.RandomCrop((size,size),padding = 4)
  test = transforms.Compose([resize,transforms.ToTensor()])
  return resize,crop,test

def raw(size):
  resize,crop,test = resize_crop_test(size)
  train_raw = transforms.Compose([resize,
                                  transforms.ToTensor()])
  return train_raw

def aug(size):
  resize,crop,test = resize_crop_test(size)
  train_aug = transforms.Compose([resize,
                                  crop,
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomVerticalFlip(),
                                  transforms.ToTensor()])
  return train_aug

def pre(size):
  resize,crop,test = resize_crop_test(size)
  train_pre = transforms.Compose([resize,
                                  transforms.GaussianBlur(2),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                  transforms.ToTensor()])
  return train_pre

def prepare_data(size):
  train = raw(size)
  resize,crop,test = resize_crop_test(size)
  trainset = torchvision.datasets.ImageFolder(root = traindir, transform = train)
  testset = torchvision.datasets.ImageFolder(root = testdir , transform = test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=35, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=35, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

# **Train, Test**

In [5]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 32
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:0.4f}")
      running_loss = 0.0

def test_epoch(model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

# **Main**

In [8]:
def main(model = 'vgg16', size = 32):
  classes = get_classes()
  datasets = prepare_data(size)
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  if model == 'vgg16':
    print("vgg16")
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
  elif model == 'resnet50':
    print("resnet50")
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True) 
  elif model == 'densenet':
    print("densenet")
    model = torchvision.models.densenet161()
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=3, bias=True)

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))



In [12]:
main('vgg16',32)  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


vgg16
Epoch 0 step 31 ave_loss 1.0983
Epoch 0 step 63 ave_loss 1.0965
Epoch 0 step 95 ave_loss 1.0762
Epoch 0 step 127 ave_loss 1.0855
Epoch 0 step 159 ave_loss 1.0964
Epoch 0 step 191 ave_loss 1.0999
              precision    recall  f1-score   support

          2C       0.25      1.00      0.41       409
          3C       0.00      0.00      0.00       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.25      1607
   macro avg       0.08      0.33      0.14      1607
weighted avg       0.06      0.25      0.10      1607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.

Epoch 1 step 31 ave_loss 1.0963
Epoch 1 step 63 ave_loss 1.0982
Epoch 1 step 95 ave_loss 1.0977
Epoch 1 step 127 ave_loss 1.0961
Epoch 1 step 159 ave_loss 1.0901
Epoch 1 step 191 ave_loss 1.0718
              precision    recall  f1-score   support

          2C       0.29      1.00      0.45       409
          3C       0.00      0.00      0.00       367
          4C       1.00      0.25      0.40       831

    accuracy                           0.39      1607
   macro avg       0.43      0.42      0.29      1607
weighted avg       0.59      0.39      0.32      1607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.

...

Epoch 7 step 31 ave_loss 0.0023
Epoch 7 step 63 ave_loss 0.0001
Epoch 7 step 95 ave_loss 0.0002
Epoch 7 step 127 ave_loss 0.0002
Epoch 7 step 159 ave_loss 0.0015
Epoch 7 step 191 ave_loss 0.0001
              precision    recall  f1-score   support

          2C       0.92      0.88      0.90       409
          3C       0.61      0.94      0.74       367
          4C       1.00      0.78      0.88       831

    accuracy                           0.84      1607
   macro avg       0.84      0.87      0.84      1607
weighted avg       0.89      0.84      0.85      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 8 step 31 ave_loss 0.0001
Epoch 8 step 63 ave_loss 0.0001
Epoch 8 step 95 ave_loss 0.0000
Epoch 8 step 127 ave_loss 0.0001
Epoch 8 step 159 ave_loss 0.0001
Epoch 8 step 191 ave_loss 0.0001
              precision    recall  f1-score   support

          2C       0.91      0.89      0.90       409
          3C       0.62      0.94      0.75       367
          4C       1.00      0.78      0.88       831

    accuracy                           0.85      1607
   macro avg       0.84      0.87      0.84      1607
weighted avg       0.89      0.85      0.85      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 step 31 ave_loss 0.0001
Epoch 9 step 63 ave_loss 0.0001
Epoch 9 step 95 ave_loss 0.0000
Epoch 9 step 127 ave_loss 0.0001
Epoch 9 step 159 ave_loss 0.0001
Epoch 9 step 191 ave_loss 0.0001
              precision    recall  f1-score   support

          2C       0.91      0.89      0.90       409
          3C       0.62      0.94      0.75       367
          4C       1.00      0.79      0.88       831

    accuracy                           0.85      1607
   macro avg       0.84      0.87      0.84      1607
weighted avg       0.89      0.85      0.86      1607

